# <b> 1. LSA(Latent Semantic Analysis)</b> 

In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import TreebankWordTokenizer

In [9]:
df_2024 = pd.read_csv('../드라마영화예능_전처리데이터/24영화_전처리.csv')
df_2023 = pd.read_csv('../드라마영화예능_전처리데이터/23영화_전처리.csv')
df_2022 = pd.read_csv('../드라마영화예능_전처리데이터/22영화_전처리.csv')

df = pd.concat([df_2024, df_2023, df_2022], axis=0).reset_index(drop=True)

df.head()

,postdate,body,description,title,preprocessed_body,preprocessed_description,preprocessed_title
0,20231219,성수동 전시 디즈니100주년 기념 팝업 House of Wish 디즈니 2024년 ...,성수동 전시 디즈니100주년 기념 팝업 House of Wish 디즈니 b2024년...,성수동 전시 디즈니100주년 기념 팝업 House of Wish 디즈니 2024년 ...,성수동 전시 디즈니 주년 기념 팝업 디즈니 위시 블로거 새틴 인플루언서새틴 일요일 ...,성수동 전시 디즈니 주년 기념 팝업 디즈니 위 시 블로거 새틴 토 수 운영 디즈니 ...,성수동 전시 디즈니 주년 기념 팝업 디즈니 위시
1,20240202,안녕하세요 죠로링입니다 저는 19년도부터 넷플릭스를 구독하여 지금까지 친구들과 아이...,이유 b2024년b 개봉 예정작 저는 계정을 공유하고 있지만 넷플릭스를 잘 보지 않...,넷플릭스 계정 공유 방법 모바일 단속 시행 2024년 넷플 개봉작 추천,안녕하세요 죠로링 년도 넷플릭스 구독 친구 아이디 공유 사용 작년 넷플릭스 금액 인...,이유 계정 공유 넷플릭스 데 넷플릭스 코리아 인스타 주,넷플릭스 계정 공유 방법 모바일 단속 시행 넷플
2,20240102,1월 개봉예정영화 2024년 최신 개봉작 한국 해외 외국 영화 추천 벌써 2일이 시...,1월 개봉예정영화 b2024년b 최신 b개봉작b 한국 해외 외국 영화 추천 벌써 2...,1월 개봉예정영화 2024년 최신 개봉작 한국 해외 외국 영화 추천,해외 외국 도 느낌 느낌 데 작년 도 내일 디즈니 위시 글 포스팅 번 김 체크 길 ...,해외 외국 도 느낌 느낌 데 작년 도,해외 외국
3,20240205,오늘은 2024년 2월 메가박스 돌비시네마 개봉작에 관한 글을 적어볼까 합니다 2월...,b2024년b 2월 7일 돌비시네마 상영작은 아가일 b2024년b 2월 7일 b개봉...,2024년 2월 메가박스 돌비시네마 개봉작 소개,은 메가박스 돌비 시네마 적 돌비 은 패스 애매 적 아이맥스 돌비 시네마 은 초 말...,돌비 시네마 상영작 아가 아가 은 돌비 시네마 포맷 가일 돌비 비전 돌비 애트모스 ...,메가박스 돌비 시네마
4,20240501,이 포스팅은 2024년 5월 개봉 예정 영화 전체를 소개하는 소개글이 아닌 저 스스...,b2024년b 5월 개봉 관람 예정 영화 리스트 흥행 괴물 형사의 독주를 막을 가정...,2024년 5월 개봉 관람 예정 영화 리스트 흥행 괴물 형사의 독주를 막을 가정의 ...,포스팅 전체 글 고민 목표 자체 고민 인하 개인 취향 선정 만 괴물 형사 독주 가정...,괴물 형사 독주 가정 달 승자 길 비수기 성수기 진입 가정 달 니,괴물 형사 독주 가정 달 승자


In [10]:
# 결측치 제거
df = df.dropna(subset=['preprocessed_body'])
df.shape

(1902, 7)

In [11]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['preprocessed_body'])

# TF-IDF 결과를 데이터프레임으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


In [15]:
# 결과 출력 함수
def print_tfidf_results(tfidf_df, document_ids, top_n=10):
    for idx in document_ids:
        print(f"Document {idx}:")
        sorted_items = sorted(tfidf_df.loc[idx].items(), key=lambda x: x[1], reverse=True)[:top_n]
        for item in sorted_items:
            if item[1] > 0:
                print(f"{item[0]}: {item[1]:.4f}")
        print("-" * 20)

# 문서 ID 리스트
document_ids = list(range(10))  # 필요한 만큼의 문서 인덱스 지정

# TF-IDF 결과 출력 (상위 10개의 TF-IDF 값만 출력)
print_tfidf_results(tfidf_df, document_ids, top_n=10)


Document 0:
공간: 0.4656
디즈니: 0.3379
팝업: 0.2042
신데렐라: 0.1940
라푼젤: 0.1795
레거시: 0.1590
백설공주: 0.1570
공주: 0.1560
주년: 0.1477
전시: 0.1416
--------------------
Document 1:
계정: 0.4131
넷플릭스: 0.3912
공유: 0.2894
인증: 0.2849
프리미엄: 0.2497
광고: 0.2064
구성원: 0.1869
방법: 0.1674
스탠다드: 0.1625
번호: 0.1608
--------------------
Document 2:
무비: 0.3893
무파사: 0.1581
쏘우: 0.1564
스파이더맨: 0.1415
소니: 0.1266
엠파이어: 0.1233
코엔: 0.1233
데드풀: 0.1206
프로즌: 0.1159
에일리언: 0.1140
--------------------
Document 3:
돌비: 0.6806
시네마: 0.4496
포맷: 0.3236
아이맥스: 0.2083
코돌비: 0.1561
아가: 0.1547
기획전: 0.1532
화면: 0.1120
용아맥: 0.1082
비율: 0.0970
--------------------
Document 4:
시사회: 0.2162
강동원: 0.1792
크래신스키: 0.1667
포켓몬스터: 0.1395
퓨리오사: 0.1382
매드: 0.1305
혹성탈출: 0.1258
동네: 0.1241
맥스: 0.1210
시사: 0.1172
--------------------
Document 5:
사진관: 0.3875
쿠첸: 0.2215
이선균: 0.1779
현수: 0.1651
전투: 0.1497
초원: 0.1472
단속: 0.1352
허구: 0.1334
차례: 0.1250
순서: 0.1191
--------------------
Document 6:
소탕: 0.2518
이동휘: 0.2292
김무열: 0.2271
범죄도시: 0.2151
마동석: 0.1900
지환: 0.1852
필리핀: 0.1791
수사대